## Recap: Data Pipeline, Network Building, Training Loop

In [1]:
import torch
from torch import nn 
from torch.utils.data import DataLoader 
from torchvision import datasets 
from torchvision.transforms import ToTensor, Lambda

### Download the data, build the data pipeline.

In [2]:
#Downloading the datasets.

train_dataset = datasets.FashionMNIST(
    root = 'data', 
    download = True, 
    transform = ToTensor(),
    train = True
)

test_dataset = datasets.FashionMNIST(
    root = 'data',
    download = True,
    train = False, 
    transform = ToTensor()
)

#Building the dataloaders. 
train_dataloader = DataLoader(train_dataset,
                             batch_size=64, 
                             shuffle=True,
                              drop_last=True, #guarantees that each batch has same size.
                             num_workers = 12
                             )

test_dataloader = DataLoader(test_dataset, 
                            batch_size=64,
                            drop_last =True,
                            num_workers = 12
                            )


In [3]:
print(train_dataset[0][0].shape)

torch.Size([1, 28, 28])


### Define the neural network.

In [4]:
class FashionClassifier(nn.Module):
    def __init__(self, img_h, img_w, hid_dim, num_classes):
        super(FashionClassifier, self).__init__()
        self.flatten = nn.Flatten()
        self.backbone = nn.Sequential(
            nn.Linear(img_h * img_w, hid_dim), 
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(hid_dim, hid_dim), 
            nn.ReLU(), 
            nn.Dropout(),
            nn.Linear(hid_dim, hid_dim), 
            nn.ReLU(), 
            nn.Dropout(),
            nn.Linear(hid_dim, hid_dim), 
            nn.ReLU(), 
            nn.Dropout(),
            nn.Linear(hid_dim, num_classes), 
        )
        
    def forward(self, X):
        h = self.flatten(X)
        out = self.backbone(h)
        return out

### Remember to push the model to the GPU 

In [5]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(device)
model = FashionClassifier(28, 28, 512, 10).to(device)

cuda:0


### Set Hyperparameters

In [6]:
learning_rate = 1e-3 
batch_size = 64 # defining here, though it should be passed to the DataLoader in initalization.
epochs = 30

### Define the Loss Function and the Optimizer

In [7]:
loss_fn = nn.CrossEntropyLoss() #this is like Keras' SparseCategoricalCrossEntropy(with_logits=True)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

### Define the Train Loop and Test Loop for each Epoch

In [8]:
def train_loop(train_dataloader, model, loss_fn, optimizer):
    model.train() # put the model to training mode in case it is not.
    num_samples = len(train_dataloader.dataset)
    epoch_loss, batches, true_preds= 0 , 0 , 0
    
    #training step.
    for (X, y) in train_dataloader:
        batches += 1
        X, y = X.to(device), y.to(device) #load the tensors to GPU.
        optimizer.zero_grad() #refresh the gradients.
        preds = model(X) #feedforward
        true_preds += (preds.argmax(dim=1) == y).type(torch.float).sum().item()
        step_loss = loss_fn(preds, y) # remember, gt is second in pytorch. it is first in TF.
        epoch_loss += step_loss.item()
        step_loss.backward() #backprop
        optimizer.step() #apply the gradients.
    
    print("Train loss: {}, Train acc: {}".format(epoch_loss/batches, true_preds / num_samples))
                
        
def test_loop(test_dataloader, model, loss_fn):
        
    model.eval() #freeze the model weights, no need to save gradients here.
    num_samples = len(test_dataloader.dataset)
    epoch_loss, batches, true_preds = 0, 0, 0 

    for X,y in test_dataloader:
        batches += 1
        X, y = X.to(device), y.to(device) # load the tensors to GPU.
        preds = model(X)
        true_preds += (preds.argmax(dim=1) == y).type(torch.float).sum().item()
        step_loss = loss_fn(preds, y)
        epoch_loss += step_loss.item()
        
    print('Validation loss: {}, Validation acc: {}'.format(epoch_loss/batches, true_preds/num_samples))        

In [9]:
for epoch in range(epochs):
    print('\n Epoch: %s \n -------------- \n '%(epoch + 1))
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
    
    


 Epoch: 1 
 -------------- 
 
Train loss: 0.7199296775406173, Train acc: 0.7299166666666667
Validation loss: 0.4758479727957493, Validation acc: 0.8253

 Epoch: 2 
 -------------- 
 
Train loss: 0.521345610965914, Train acc: 0.8162333333333334
Validation loss: 0.4462082364047185, Validation acc: 0.8388

 Epoch: 3 
 -------------- 
 
Train loss: 0.4826131201445611, Train acc: 0.8296833333333333
Validation loss: 0.41621488695725417, Validation acc: 0.8477

 Epoch: 4 
 -------------- 
 
Train loss: 0.4672902920139256, Train acc: 0.8372166666666667
Validation loss: 0.4297080136453494, Validation acc: 0.8483

 Epoch: 5 
 -------------- 
 
Train loss: 0.44698744970935383, Train acc: 0.8424333333333334
Validation loss: 0.3878153860569, Validation acc: 0.8583

 Epoch: 6 
 -------------- 
 
Train loss: 0.43748718283855803, Train acc: 0.8466166666666667
Validation loss: 0.3881936283447804, Validation acc: 0.8576

 Epoch: 7 
 -------------- 
 
Train loss: 0.43627187633527126, Train acc: 0.847683